In [15]:
import pandas as pd
import numpy as np
import sklearn.datasets

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [30]:
#Please change to your name.
name = "Michael Harris"

#Please use this to control EDA. 
show_eda = False

In [12]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
167,0,0.692,0.674,0.232,0.982,0.650,0.328,0.936,0.127,0.693,...,0.692,0.158,0.207,0.155,0.707,0.803,0.450,0.143,0.845,0.333
98,0,0.640,0.559,0.887,0.587,0.626,0.528,0.451,0.738,0.894,...,0.696,0.130,0.839,0.114,0.766,0.322,0.307,0.825,0.539,0.532
86,1,0.004,0.938,0.339,0.466,0.281,0.692,0.578,0.500,0.179,...,0.951,0.695,0.152,0.146,0.757,0.085,0.809,0.948,0.290,0.921
45,1,0.527,0.432,0.164,0.568,0.747,0.474,0.187,0.303,0.381,...,0.566,0.369,0.069,0.086,0.370,0.446,0.623,0.310,0.284,0.195
81,1,0.028,0.706,0.018,0.984,0.564,0.831,0.697,0.032,0.414,...,0.085,0.511,0.551,0.914,0.061,0.850,0.977,0.557,0.624,0.935


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [88]:
y = np.array(df["target"]).reshape(-1,1)
X = np.array(df.drop(columns={"target"}))
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

In [118]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'DT__max_depth': [None, 10, 20],
    'DT__min_samples_split': [2, 5, 10],
    'DT__min_samples_leaf': [1, 2, 5]
}

grid_search = GridSearchCV(best, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
best_tuned = grid_search.best_estimator_

Best parameters: {'DT__max_depth': 10, 'DT__min_samples_leaf': 1, 'DT__min_samples_split': 5}


In [188]:
clf = DecisionTreeClassifier(min_samples_split=5, min_samples_leaf=2)
clf = clf.fit(X_train, y_train)
print(clf.get_depth())
print("Training Accuracy:", clf.score(X_train, y_train))
print("Testing Accuracy:", clf.score(X_test, y_test))

7
Training Accuracy: 0.9625668449197861
Testing Accuracy: 0.5555555555555556


In [189]:
#Build pipeline
pipeline_steps = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(min_samples_split=5, min_samples_leaf=2))
    ]

best = Pipeline(pipeline_steps)

best.fit(X_train, y_train)

print(best.score(X_train, y_train))
print(best.score(X_test, y_test))
   

0.9625668449197861
0.5555555555555556


### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [190]:
print(best.score(X_test, y_test))
print(best)

0.5555555555555556
Pipeline(steps=[('scaler', StandardScaler()),
                ('DT',
                 DecisionTreeClassifier(min_samples_leaf=2,
                                        min_samples_split=5))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [196]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.5482513933393822
0.5482025316455696
Michael Harris 0.5482269624924759


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li>
<li>
</ul>